# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

X. Zhang  ->  X. Zhang  |  ['X. Zhang']
L. Xie  ->  Z.-L. Xie  |  ['L. Xie']
F. Walter  ->  F. Walter  |  ['F. Walter']


Arxiv has 73 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2509.03558
extracting tarball to tmp_2509.03558...

 done.
Retrieving document from  https://arxiv.org/e-print/2509.03585
extracting tarball to tmp_2509.03585... done.


L. Xie  ->  Z.-L. Xie  |  ['L. Xie']


Found 72 bibliographic references in tmp_2509.03585/aa54459-25.bbl.
Retrieving document from  https://arxiv.org/e-print/2509.03831
extracting tarball to tmp_2509.03831... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.03585-b31b1b.svg)](https://arxiv.org/abs/2509.03585) | **The clustering of C IV and Si IV at the end of reionisation: A perspective from the E-XQR-30 survey**  |
|| L. Welsh, et al. -- incl., <mark>L. Xie</mark> |
|*Appeared on*| *2025-09-05*|
|*Comments*| *9 pages, 6 figures, accepted for publication in Astronomy & Astrophysics*|
|**Abstract**|            We aim to study the clustering of metal absorption lines and the structures that they arise in as a function of cosmic time. We focus on C IV and Si IV absorption features that are identified along a given quasar sightline. We exploit the two-point correlation function (2PCF) to investigate the clustering of these structures as a function of their separation. We utilise the E-XQR-30 data to perform a novel analysis at z>5. We also draw on literature surveys (including XQ-100) of lower redshift quasars to investigate the possible evolution of this clustering towards cosmic noon (i.e., z~2-3). We find no significant evolution with redshift when considering the separation of absorbers in velocity space. Since we are comparing data across a large interval of cosmic time, we also consider the separation between absorbers in the reference frame of physical distances. In this reference frame, we find that the amplitude of the clustering increases with cosmic time for both C IV and Si IV on the scales of <1500 physical kpc. For the first time, we assess the 2PCF of C IV and Si IV close to the epoch of reionisation utilising the absorber catalogue from the E-XQR-30 survey. We compare this with lower redshift data and find that, on small scales, the clustering of these structures grows with cosmic time. We compare these results to the clustering of galaxies in the GAEA simulations. It appears that the structures traced by C IV are broadly comparable to the galaxies from the considered simulations. The clustering is most similar to that of the galaxies with virial masses M~10^10.5 M_sun. We require tailor-made simulations to investigate the full range of factors contributing to the observed clustering. Future ground-based spectrographs will further facilitate surveys of absorbers at this epoch with increased sensitivity.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.03558-b31b1b.svg)](https://arxiv.org/abs/2509.03558) | **Seeing and Meteorological Analysis at the North-1 and North-2 Points of Muztagh-Ata Site on the Pamir Plateau**  |
|| W. Gu, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-09-05*|
|*Comments*| *14 pages,26 figures. published in MNRAS*|
|**Abstract**|            To support the selection of large optical/infrared telescope sites in western China, long-term monitoring of atmospheric conditions and astronomical seeing has been conducted at the Muztagh-Ata site on the Pamir Plateau since 2017. With the monitoring focus gradually shifting northward, three stations were established: the South Point, North-1 point, and North-2 point. The North-1 point,selected as the site for the 1.93m Muztagh Optical Survey Telescope (MOST), has recorded seeing and meteorological parameters since late 2018. In 2023,the North-2 point was established approximately 1.5 km northeast of North-1 point as a candidate location for a future large-aperture telescope. A 10m DIMM tower and a PC-4A environmental monitoring system were installed to evaluate site quality. This study presents a comparative analysis of data from the North-1 and North-2 points during this http URL median seeing is 0.89 arcsecs at North-1 and 0.78 arcsecs at North-2. Both points show clear seasonal and diurnal variations,with winter nights offering optimal observing this http URL average, about 64% of the nighttime duration per year is suitable for astronomical observations. Nighttime temperature variation is low :2.03 at North-1 and 2.10 at North-2 .Median wind speeds are 5-6 m/s, with dominant directions between 210 and 300, contributing to stable airflow. Moderate wind suppresses turbulence, while strong shear and rapid fluctuations degrade image quality. These findings confirm that both the North-1 and North-2 points offer high-quality atmospheric conditions and serve as promising sites for future ground-based optical/infrared telescopes in western China.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.03831-b31b1b.svg)](https://arxiv.org/abs/2509.03831) | **Discovery of the pre-main-sequence eclipsing binary MML 48**  |
|| Y. G. M. Chew, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2025-09-05*|
|*Comments*| *13 pages, 11 figures, accepted for publication in A&A*|
|**Abstract**|            We present the discovery of the eclipsing binary MML 48, which is a member of Upper Centaurus Lupus, has an associated age of 16 Myr, and is composed of two young, low-mass stars. We used space- and ground-based observations to characterize the system with both time-series photometry and spectroscopy. Given the extreme mass ratio between the stars, q_EB = 0.209 +- 0.014, we modeled a single-lined spectroscopic and eclipsing binary system. The orbital period, 2.0171068 +- 0.0000004 d, is measured from the highest precision light curves. We derive a primary mass of 1.2 +- 0.07 Msun using stellar models, and with radial velocities we measured a secondary mass of 0.2509 +- 0.0078 Msun. The radii are large, as expected for pre-main-sequence stars, and are measured as 1.574 +- 0.026 +- 0.050 Rsun and 0.587 +- 0.0095 +- 0.050 Rsun, for the primary and secondary stars, respectively. MML 48 joins the short list of known low-mass, pre-main-sequence eclipsing binaries (EBs), being one of only five systems with intermediate ages (15-25 Myr), and the system with the most extreme mass ratio. The primary star is currently at the "fusion bump", undergoing an over-production of energy in the core due to the build-up of 3He before reaching its equilibrium abundance set by the proton-proton (p-p) I chain. MML 48 A is the first young star in an eclipsing system that has been found during its fusion bump. MML 48 is thus an important benchmark for low-mass stellar evolution at a time when the stars are rapidly changing, which allows for a tight constraint on the corresponding isochrone given the uneven mass ratio.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2509.03585/./Figures/2PCF_1000_LP_umerged_XQR30_scales_tot_new_chains_c63_step.png', 'tmp_2509.03585/./Figures/fin_2PCF_merged_b2_complete_zsplit_w_sim_mpc_civ_sys_new_kpc_conv_steps.png', 'tmp_2509.03585/./Figures/2PCF_complete_1000_2pan_kms_mult_bins_tot_new_chains_step.png']
copying  tmp_2509.03585/./Figures/2PCF_1000_LP_umerged_XQR30_scales_tot_new_chains_c63_step.png to _build/html/
copying  tmp_2509.03585/./Figures/fin_2PCF_merged_b2_complete_zsplit_w_sim_mpc_civ_sys_new_kpc_conv_steps.png to _build/html/
copying  tmp_2509.03585/./Figures/2PCF_complete_1000_2pan_kms_mult_bins_tot_new_chains_step.png to _build/html/
exported in  _build/html/2509.03585.md
    + _build/html/tmp_2509.03585/./Figures/2PCF_1000_LP_umerged_XQR30_scales_tot_new_chains_c63_step.png
    + _build/html/tmp_2509.03585/./Figures/fin_2PCF_merged_b2_complete_zsplit_w_sim_mpc_civ_sys_new_kpc_conv_steps.png
    + _build/html/tmp_2509.03585/./Figures/2PCF_complete_1000_2pan_kms_mult_bins_tot_new

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\xqr}{E-XQR-30}$
$\newcommand{\fe}[1]{Fe {\sc ii}~\lambda#1}$
$\newcommand{\civ}{C {\sc iv}}$
$\newcommand{\siiv}{Si {\sc iv}}$
$\newcommand{\cf}{\xi(v, v+\Delta v)}$
$\newcommand{\md}{\Omega_{\rm C \textsc{iv}}}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\comment}[1]{\textcolor{blue}{#1} }$</div>



<div id="title">

# The clustering of $\civ$ and $\siiv$ at the end of reionisation: A perspective from the E-XQR-30 survey

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.03585-b31b1b.svg)](https://arxiv.org/abs/2509.03585)<mark>Appeared on: 2025-09-05</mark> -  _9 pages, 6 figures, accepted for publication in Astronomy & Astrophysics_

</div>
<div id="authors">

L. Welsh, et al. -- incl., <mark>L. Xie</mark>

</div>
<div id="abstract">

**Abstract:**            We aim to study the clustering of metal absorption lines and the structures that they arise in as a function of cosmic time. We focus on C IV and Si IV absorption features that are identified along a given quasar sightline. We exploit the two-point correlation function (2PCF) to investigate the clustering of these structures as a function of their separation. We utilise the E-XQR-30 data to perform a novel analysis at z>5. We also draw on literature surveys (including XQ-100) of lower redshift quasars to investigate the possible evolution of this clustering towards cosmic noon (i.e., z~2-3). We find no significant evolution with redshift when considering the separation of absorbers in velocity space. Since we are comparing data across a large interval of cosmic time, we also consider the separation between absorbers in the reference frame of physical distances. In this reference frame, we find that the amplitude of the clustering increases with cosmic time for both C IV and Si IV on the scales of <1500 physical kpc. For the first time, we assess the 2PCF of C IV and Si IV close to the epoch of reionisation utilising the absorber catalogue from the E-XQR-30 survey. We compare this with lower redshift data and find that, on small scales, the clustering of these structures grows with cosmic time. We compare these results to the clustering of galaxies in the GAEA simulations. It appears that the structures traced by C IV are broadly comparable to the galaxies from the considered simulations. The clustering is most similar to that of the galaxies with virial masses M~10^10.5 M_sun. We require tailor-made simulations to investigate the full range of factors contributing to the observed clustering. Future ground-based spectrographs will further facilitate surveys of absorbers at this epoch with increased sensitivity.         

</div>

<div id="div_fig1">

<img src="tmp_2509.03585/./Figures/2PCF_1000_LP_umerged_XQR30_scales_tot_new_chains_c63_step.png" alt="Fig2" width="100%"/>

**Figure 2. -** Top: The two point correlation function of $\civ$ and $\siiv$ based on the UVES LP data previously analysed in [Calura, Cristiani and Viel (2010)](), [Finlator and Cristiani (2022)](). In this  calculation we have imposed the same column density cuts as applied to the $\xqr$ data. Compared to $\xqr$, the resulting distributions are sampled more frequently, with an additional  evaluation point at $\Delta v = 150$ $\kms$. The subsequent bins are consistent with those used for XQR-30. Bottom: The 2PCF of $\civ$ and $\siiv$ shown on the smallest scale accessible based on the limited statistics of the $\siiv$ data. These bins span $\Delta v=$[$50-300, 300-600, 600-1200, 1200-1500, 1500-3000$] $\kms$ for the first 5 bins. Note that distributions of $\xi$ values are formed via many realisations of the randomly distributed lines. For each sample, the solid line represents the median value while the shaded region encompasses the interquartile range. (*fig:2pcf_xqr30*)

</div>
<div id="div_fig2">

<img src="tmp_2509.03585/./Figures/fin_2PCF_merged_b2_complete_zsplit_w_sim_mpc_civ_sys_new_kpc_conv_steps.png" alt="Fig4" width="100%"/>

**Figure 4. -** The 3D real space correlation calculated at different epochs using the GAEA simulation suite snapshots. The correlation is calculated using the corrfunc package. The different colors correspond to galaxies with different virial parent halo masses. The top and bottom panels correspond to different epochs as indicated by the titles. The solid curves represent the clustering of all galaxies in the snapshot. The dotted lines represent that of only the central galaxies associated with halos. The observed 2PCF of $\civ$ and $\siiv$ have been overplotted on these simulation data across different epochs. In this case, we calculate the 2PCF of $\civ$ systems (i.e., all the absorption within 200 $\kms$ is considered one feature). Again, the shaded regions correspond to the interquartile range of the observed distributions.  (*fig:xi_sim_n_obs*)

</div>
<div id="div_fig3">

<img src="tmp_2509.03585/./Figures/2PCF_complete_1000_2pan_kms_mult_bins_tot_new_chains_step.png" alt="Fig3" width="100%"/>

**Figure 3. -** The 2PCF of $\civ$(top) and $\siiv$(bottom) based on the absorbers in the $\xqr$ survey now combined with that of the lower redshift `UVES LR' data. The UVES LR data are taken with higher resolution and therefore we have merged the features that would not be resolved with XSHOOTER. We additionally include the 2PCF of $\siiv$ from a third dataset, the XQ-100 survey. The shaded region encompasses the interquartile range of the respective distributions. The $\siiv$ absorbers in XQ-100 are found at an intermediate epoch between the XQR-30 and UVES LR data. Note that we can probe down to smaller separations (within 200 $\kms$) with $\civ$ due to the increased statistics.  (*fig:2pcf_merged*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.03585"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

130  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
